# This notebook formats the data, and runs it through our bidirectional transformer architecture.

### Data formatting:

Refer to this notebook: https://github.com/ocatak/lstm_malware_detection/blob/master/deep_learnin_lstm_malware_detection.ipynb

### Bidirectional architecture:

... to be added

In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.insert(0, './keras_bert_rec')
import importlib
import keras_bert_rec as rec
importlib.reload(rec)

import argparse
import keras
import numpy as np
import pandas as pd
import pickle
import sys
import tensorflow as tf
import importlib

from itertools import chain
from keras import backend as K
from keras.models import load_model, Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM, Dense, Dropout, Embedding
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import SpatialDropout1D

Using TensorFlow backend.


## Formatting data

In [2]:
DATA_PATH = "../data/"

In [3]:
# Read in data
## Calls
malware_calls_df = pd.read_csv(f"{DATA_PATH}calls.zip", compression="zip",
                               sep="\t", names=["API_Calls"])
## Labels ("types")
malware_labels_df = pd.read_csv(f"{DATA_PATH}types.zip", compression="zip",
                               sep="\t", names=["API_Labels"])

In [4]:
## Concat data
malware_calls_df["API_Labels"] = malware_labels_df.API_Labels
malware_calls_df["API_Calls"] = malware_calls_df.API_Calls.apply(lambda x: " ".join(x.split(",")))

malware_calls_df["API_Labels"] = malware_calls_df.API_Labels.apply(lambda x: 1 if x == "Virus" else 0)

In [5]:
max_words = 800
max_len = 100

X = malware_calls_df.API_Calls
Y = malware_calls_df.API_Labels.astype('category').cat.codes

In [6]:
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X)
print('Found %s unique tokens.' % len(tok.word_index))
X = tok.texts_to_sequences(X.values)
X = sequence.pad_sequences(X, maxlen=max_len)
print('Shape of data tensor:', X.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=0.15)

le = LabelEncoder()
Y_train_enc = le.fit_transform(Y_train)
Y_train_enc = np_utils.to_categorical(Y_train_enc)

Y_test_enc = le.transform(Y_test)
Y_test_enc = np_utils.to_categorical(Y_test_enc)

Found 278 unique tokens.
Shape of data tensor: (7107, 100)


NameError: name 'LabelEncoder' is not defined

## Bidirectional model